In [ ]:
#default_exp expanding

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Expanding

> Expanding window operations

In [ ]:
#hide
import random

from nbdev.showdoc import *

In [ ]:
#export
from math import sqrt
from typing import Callable, Optional

import numpy as np
from numba import njit

from window_ops.rolling import *
from window_ops.utils import _gt, _lt

In [ ]:
import pandas as pd

np.random.seed(0)
array = np.random.rand(100)
series = pd.Series(array)

In [ ]:
#exporti
def _expanding_op(rolling_op: Callable, x: np.ndarray, min_samples: int = 1) -> np.ndarray:
    n_samples = x.size
    return rolling_op(x, window_size=n_samples, min_samples=min_samples)

In [ ]:
#export
def expanding_mean(x: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_mean, x)

In [ ]:
np.testing.assert_allclose(expanding_mean(array), series.expanding().mean().values)

In [ ]:
#export
def expanding_std(x: np.ndarray) -> np.ndarray:
    return _expanding_op(rolling_std, x, min_samples = 2)

In [ ]:
np.testing.assert_allclose(expanding_std(array), series.expanding().std().values)

In [ ]:
#exporti
@njit
def _expanding_comp(comp: Callable,
                    x: np.ndarray) -> np.ndarray:
    n_samples = x.size
    out = np.empty(n_samples, dtype=np.float32)
    pivot = x[0]
    out[0] = pivot
    for i in range(1, n_samples):
        if comp(x[i], pivot) > 0:
            pivot = x[i]
        out[i] = pivot
    return out

In [ ]:
#export
def expanding_max(x: np.ndarray) -> np.ndarray:
    return _expanding_comp(_gt, x)

In [ ]:
np.testing.assert_allclose(expanding_max(array), series.expanding().max())

In [ ]:
#export
def expanding_min(x: np.ndarray) -> np.ndarray:
    return _expanding_comp(_lt, x)

In [ ]:
np.testing.assert_allclose(expanding_min(array), series.expanding().min())

## Seasonal

In [ ]:
idxs = [i % 7 for i in range(array.size)]
seasons = np.arange(7)[idxs]
grouped_series = series.groupby(seasons)

In [ ]:
#exporti
def _seasonal_expanding_op(rolling_op: Callable, x: np.ndarray, season_length: int,
                           min_samples: int = 1) -> np.ndarray:
    n_samples = x.size
    return rolling_op(x, season_length=season_length, window_size=n_samples, min_samples=min_samples)

In [ ]:
#export
def seasonal_expanding_mean(x: np.ndarray, season_length: int) -> np.ndarray:
    return _seasonal_expanding_op(seasonal_rolling_mean, x, season_length)

In [ ]:
np.testing.assert_allclose(seasonal_expanding_mean(array, 7), grouped_series.transform(lambda y: y.expanding().mean()))

In [ ]:
#export
def seasonal_expanding_std(x: np.ndarray, season_length: int) -> np.ndarray:
    return _seasonal_expanding_op(seasonal_rolling_std, x, season_length, min_samples=2)
    return out

In [ ]:
np.testing.assert_allclose(seasonal_expanding_std(array, 7), grouped_series.transform(lambda y: y.expanding().std()))

In [ ]:
#export
def seasonal_expanding_min(x: np.ndarray, season_length: int):
    return _seasonal_expanding_op(seasonal_rolling_min, x, season_length)

In [ ]:
np.testing.assert_allclose(seasonal_expanding_min(array, 7), grouped_series.transform(lambda y: y.expanding().min()))

In [ ]:
#export
def seasonal_expanding_max(x: np.ndarray, season_length: int):
    return _seasonal_expanding_op(seasonal_rolling_max, x, season_length)

In [ ]:
np.testing.assert_allclose(seasonal_expanding_min(array, 7), grouped_series.transform(lambda y: y.expanding().max()).values,)